In [24]:
from ecostyles import EcoStyles
# Create styles instance
styles = EcoStyles()

import altair as alt
import pandas as pd

In [25]:
# Register and enable a theme
styles.register_and_enable_theme(theme_name="article")

In [26]:
# Load average electricity data (excl CCL)
elec_excl_df = pd.read_excel('elec_uk_non-dom.xlsx', skiprows=6, sheet_name='3.4.1 (excl CCL)')
elec_excl_df = elec_excl_df[['Year', 'Quarter', 'Electricity: Average (Pence per kWh)']]
elec_excl_df = elec_excl_df.rename(columns={'Electricity: Average (Pence per kWh)': 'Electricity excl CCL'})
elec_excl_df['Quarter'] = elec_excl_df['Quarter'].str.replace(r'(st|nd|rd|th)$', '', regex=True)
elec_excl_df

,Year,Quarter,Electricity excl CCL
0,2004,1,3.724000
1,2004,2,3.593000
2,2004,3,3.663000
3,2004,4,4.456000
4,2005,1,4.572000
...,...,...,...
82,2024,3,25.147581
83,2024,4,24.975982
84,2025,1,25.334168
85,2025,2,23.675360


In [27]:
# Load average electricity data (incl CCL)
elec_incl_df = pd.read_excel('elec_uk_non-dom.xlsx', skiprows=6, sheet_name='3.4.2 (incl CCL)')
elec_incl_df = elec_incl_df[['Year', 'Quarter', 'Electricity: Average (Pence per kWh)']]
elec_incl_df = elec_incl_df.rename(columns={'Electricity: Average (Pence per kWh)': 'Electricity incl CCL'})
elec_incl_df['Quarter'] = elec_incl_df['Quarter'].str.replace(r'(st|nd|rd|th)$', '', regex=True)
elec_incl_df

,Year,Quarter,Electricity incl CCL
0,2004,1,4.015000
1,2004,2,3.887000
2,2004,3,3.946000
3,2004,4,4.742000
4,2005,1,4.845000
...,...,...,...
82,2024,3,25.669576
83,2024,4,25.503655
84,2025,1,25.867873
85,2025,2,24.198340


In [28]:
# Merge and reformat dataframes
avg_elec_df = elec_excl_df.merge(elec_incl_df, on=['Year','Quarter'], how='outer')
avg_elec_df = avg_elec_df.melt(
    id_vars = ['Year', 'Quarter'],
    value_vars = ['Electricity excl CCL', 'Electricity incl CCL'],
    var_name = 'Type',
    value_name = 'Electricity'
)
avg_elec_df = avg_elec_df.sort_values(['Year', 'Quarter'])
avg_elec_df['Period'] = 'Q' + avg_elec_df['Quarter'].astype(str) + ' ' + avg_elec_df['Year'].astype(str)
avg_elec_df

,Year,Quarter,Type,Electricity,Period
0,2004,1,Electricity excl CCL,3.724000,Q1 2004
87,2004,1,Electricity incl CCL,4.015000,Q1 2004
1,2004,2,Electricity excl CCL,3.593000,Q2 2004
88,2004,2,Electricity incl CCL,3.887000,Q2 2004
2,2004,3,Electricity excl CCL,3.663000,Q3 2004
...,...,...,...,...,...
171,2025,1,Electricity incl CCL,25.867873,Q1 2025
85,2025,2,Electricity excl CCL,23.675360,Q2 2025
172,2025,2,Electricity incl CCL,24.198340,Q2 2025
86,2025,3,Electricity excl CCL,22.955889,Q3 2025


In [66]:
# Plot line chart
elec_line_chart = alt.Chart(avg_elec_df).mark_line().encode(
    x = alt.X('Period:N',
              sort=None,
              axis=alt.Axis(
                  values=['Q4 2004', 'Q4 2006', 'Q4 2008', 'Q4 2010', 'Q4 2012', 'Q4 2014', 'Q4 2016', 'Q4 2018', 'Q4 2020', 'Q4 2022', 'Q4 2024'],
                  grid=False,
                  titleFontSize=13,
                  labelFontSize=13
                  )),
    y = alt.Y('Electricity:Q', 
              title = 'Pence per kWh', 
              axis=alt.Axis(
                  titleFontSize=13,
                  labelFontSize=13,
                  grid=False)),
    color = alt.Color('Type:N',legend=alt.Legend(labelFontSize=13))
)
styles.add_source(elec_line_chart, 'ONS - Prices of fuels purchased by non-domestic consumers in the UK')
elec_line_chart

alt.Chart(...)

In [30]:
# Load industrial electricity prices data
iea_elec_df = pd.read_excel('ind_elec_price.xlsx', skiprows=8, sheet_name='5.3.1 (inc. taxes)')
iea_elec_df = iea_elec_df.drop(['UK relative to IEA median%','UK relative to IEA rank','UK relative to G7 rank','Japan','Australia','Netherlands','Denmark'], axis=1)
iea_elec_df = iea_elec_df[iea_elec_df['Year'] == 2024]
iea_elec_df = iea_elec_df.melt(
    id_vars=['Year'],
    var_name='Country',
    value_name='Value'
)
iea_elec_df

,Year,Country,Value
0,2024,Austria,18.161283
1,2024,Belgium,18.140935
2,2024,Finland,8.141674
3,2024,France,16.331674
4,2024,Germany,20.993022
5,2024,Greece,15.242217
6,2024,Ireland,24.768391
7,2024,Italy,23.522087
8,2024,Luxembourg,15.104022
9,2024,Portugal,12.406239


In [31]:
# Sort countries by value
iea_elec_df = iea_elec_df.sort_values('Value', ascending=False)

# Get countries excluding median
countries = iea_elec_df[iea_elec_df['Country'] != 'IEA median']['Country'].tolist()

# Create sort order: countries (high to low) + gap + median
sort_order = countries + ['___gap1___','___gap2___', 'IEA median']

# Add gap rows
gap_rows = pd.DataFrame({
    'Year': [2024, 2024],
    'Country': ['___gap1___','___gap2___'],
    'Value': [0, 0]
})
iea_elec_df = pd.concat([iea_elec_df, gap_rows], ignore_index=True)

iea_elec_df

,Year,Country,Value
0,2024,United Kingdom,26.63
1,2024,Ireland,24.768391
2,2024,Switzerland,24.428182
3,2024,Italy,23.522087
4,2024,Germany,20.993022
5,2024,Czech Republic,20.576917
6,2024,Poland,20.036201
7,2024,Slovakia,19.69
8,2024,Hungary,18.251994
9,2024,Austria,18.161283


In [67]:
# Plot bar chart
iea_bar_chart = alt.Chart(iea_elec_df).mark_bar().encode(
    y=alt.Y('Country:N', 
            sort=sort_order,
            axis=alt.Axis(
                labelExpr='indexof(datum.label, "___gap") >= 0 ? "" : datum.label',
                labelFontSize=13,
                grid=False
            ),
            title=''),
    x=alt.X('Value:Q', title='Pence per kWh', axis=alt.Axis(titleFontSize=13, labelFontSize=13, grid=False)),
    opacity=alt.condition(
        'indexof(datum.Country, "___gap") >= 0',
        alt.value(0),
        alt.value(1)
    )
)

styles.add_source(iea_bar_chart, 'DESNZ - Industrial electricity prices in the IEA; US prices from EIA and Norway prices from SSB, converted to pence using annual average exchange rates')
iea_bar_chart

alt.Chart(...)

In [58]:
# Load sme survey data
sme_barrier_df = pd.read_excel('sme_survey_2024.xlsx', sheet_name='Tables', skiprows=1105, nrows=15)
sme_barrier_df = sme_barrier_df.rename(columns={'Sample size, business type': 'Response'})
sme_barrier_df = sme_barrier_df[['Response', 'Total']]
sme_barrier_df['Response'] = sme_barrier_df['Response'].str.replace(' Col %', '', regex=False)
sme_barrier_df = sme_barrier_df[~sme_barrier_df['Response'].str.contains('Bases', na=False)]
sme_barrier_df = sme_barrier_df[~sme_barrier_df['Response'].isin(['Don\'t know', 'Refused'])]
response_mapping = {
    'You thought you would be rejected': 'Thought would be rejected',
    'You thought it would be too expensive': 'Too expensive',
    "You don't want to take on additional risk": 'Don\'t want additional risk',
    'Now is not the right time because of economic conditions': 'Wrong time due to economy',
    "You didn't know where to find the appropriate finance you needed": 'Unsure where to find finance',
    'Poor credit history': 'Poor credit history',
    'The decision would have taken too long/too much hassle': 'Too long/too much hassle',
    'Bank forms and literature hard to understand': 'Forms hard to understand',
    "You thought you'd be asked to provide too much security": 'Too much security required'
}
sme_barrier_df['Response'] = sme_barrier_df['Response'].replace(response_mapping)
sme_barrier_df = sme_barrier_df.sort_values('Total', ascending=False)
sme_barrier_df

,Response,Total
4,Don't want additional risk,63
3,Too expensive,60
5,Wrong time due to economy,59
2,Thought would be rejected,34
8,Too long/too much hassle,29
10,Too much security required,28
6,Unsure where to find finance,22
11,Other,18
9,Forms hard to understand,17
7,Poor credit history,10


In [68]:
# Plot bar chart
sme_barrier_bar_chart = alt.Chart(sme_barrier_df).mark_bar().encode(
    y=alt.Y('Response:N',
            sort=None, 
            title='Reasons for not seeking additional finance in last 12 months',
            axis=alt.Axis(
                titleAlign='left',
                titleAnchor='start',
                titleAngle=0,
                titleFontSize=13, 
                labelFontSize=13, 
                grid=False)),
    x=alt.X('Total:Q',
            title='Proportion of businesses that needed finance but did not apply',
            axis=alt.Axis(titleFontSize=13, labelExpr='datum.value + "%"', labelFontSize=13, grid=False))
)
styles.add_source(sme_barrier_bar_chart, 'DBT - Small Business Survey 2024')
sme_barrier_bar_chart

alt.Chart(...)

In [ ]:
# Save charts
styles.save(elec_line_chart, 'visualisation', 'elec_line_chart', width=450, height=360)
styles.save(iea_bar_chart, 'visualisation', 'iea_bar_chart', width=450, height=360)
styles.save(sme_barrier_bar_chart, 'visualisation', 'sme_barrier_bar_chart', width=450, height=360)